### This notebook is used mainly for the Capstone Project on Location Data, part of IBM's Coursera online course.

Done by: Munish Kumar

#### import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import re
import requests
import pickle
import seaborn as sns
import os
import glob
sns.set()

from functools import reduce
from datetime import timedelta, date

In [2]:
# Start, test
print('Hello Capstone Project Course!')

start = time.time()

Hello Capstone Project Course!


Declare some global variables

In [3]:
dir_name = 'C:/Users/quant/Anaconda3/@DATA/Coursera'
filename_suffix = 'csv'

#### Method 1: Web Scrape with Pandas the table present in wikipedia, using the read_html method for pandas

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal Code   180 non-null    object
 1   Borough       180 non-null    object
 2   Neighborhood  103 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


The groupby method will be used on the 'Postal Code' column to arrange the data as shown in the table. 

Will drop all cells that are 'Not Assigned' and recreate the index

In [6]:
df_new = df.groupby(df['Postal Code']).sum()
mask = df_new[df_new['Borough']!='Not assigned']
mask = mask.rename_axis('PostalCode').reset_index()

count = 0

for i in range(len(df_new['Borough'])):
    if df_new['Neighborhood'][i] == 'Not assigned':
        df_new['Neighborhood'][i] = df_new['Borough']
        count = i
print('Number of Neighbourhoods which were not assigned: ', count)
print('------------------------------------------------------------------')

print('----------------------Information on Table------------------------')
print(mask.info())
print('------------------------------------------------------------------')

print('----------------------Statistics on Table-----------------------\n', 
      mask.describe(include = 'all'))
print('------------------------------------------------------------------')

print('--------------------------Shape of Table------------------------\n', 
      mask.shape)
print('------------------------------------------------------------------')

mask

Number of Neighbourhoods which were not assigned:  0
------------------------------------------------------------------
----------------------Information on Table------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PostalCode    103 non-null    object
 1   Borough       103 non-null    object
 2   Neighborhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB
None
------------------------------------------------------------------
----------------------Statistics on Table-----------------------
        PostalCode     Borough Neighborhood
count         103         103          103
unique        103          10           99
top           M4W  North York    Downsview
freq            1          24            4
------------------------------------------------------------------
--------------------------Shape of Tabl

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


From the Statistics above, it looks like all neighbourhoods have been assigned a Borough

In [7]:
# This bit of code was taking way too long, and I wasnt sure if it
# was the way I wrote the code, or if the issues with geocoder
# I decided to just leave this cell commented out, but 
# this code should be revisited at some point

#import geocoder # import geocoder

#def get_geocoder(df_pc):
#    # initialize your variable to None
#    lat_lng_coords = None

#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(df_pc))
#      lat_lng_coords = g.latlng

#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
    
# mask['Latitude'], mask['Longitude'] =\
# zip(*mask['PostalCode'].apply(get_geocoder(mask)))

I originally tried using geoencoder, but the code above was running for an abnormally long time. I am not sure how to trouble shoot this, as I dont know if this is due to my code or due to issues with geoencoder. For simplicity, I just used the excel document provided online to complete the table. 

In [8]:
df_new = pd.read_csv('C:/Users/quant/Anaconda3/@DATA/Coursera/Geospatial_Coordinates.csv')
df_new

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
mask['Latitude']=np.where(df_new['Postal Code'] == mask['PostalCode'],
                         df_new['Latitude'],
                         0)
mask['Longitude']=np.where(df_new['Postal Code'] == mask['PostalCode'],
                         df_new['Longitude'],
                         0)
print(mask.isnull().sum())
mask

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Now, I will use folium to display the above areas in the dataframe above

In [10]:
mask.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [19]:
import folium
from geopy.geocoders import Nominatim

address = 'Toronto, CA'

geolocator = Nominatim(user_agent = 'toroto_expolorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Geographical Coordinates of Toronto are {}, {}."\
      .format(latitude, longitude)
     )

# Generate map centered around the area
loc_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add the zones above as red circle
for lat, lng, borough, neighbourhood in zip(mask['Latitude'], 
    mask['Longitude'],mask['Borough'],mask['Neighborhood']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng], radius = 5, popup=label, color = 'red', 
        fill_color = '#3186cc', fill_opacity = 0.7, fill = True,
        parse_html=False).add_to(loc_map)

loc_map

Geographical Coordinates of Toronto are 43.6534817, -79.3839347.


Now, I will only focus on one area which is Downtown Toronto itself

In [20]:
DT_data = mask[mask['Borough']=='Downtown Toronto'].reset_index(drop=True)
DT_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [34]:
city = 'Toronto, CA'
borough=['Downtown Toronto']

for i in borough:
    address = i + ', ' + city
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('Geographical Coordinates of ', address + ' is {}, {}.'\
      .format(latitude, longitude)
     )

Geographical Coordinates of  Downtown Toronto, Toronto, CA is 43.6563221, -79.3809161.


In [33]:
# Generate map centered around the area
DT_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add the zones above as red circle
for lat, lng, borough, neighbourhood in zip(DT_data['Latitude'], 
    DT_data['Longitude'],DT_data['Borough'],DT_data['Neighborhood']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng], radius = 5, popup=label, color = 'red', 
        fill_color = '#3186cc', fill_opacity = 0.7, fill = True,
        parse_html=False).add_to(DT_map)

DT_map

### Extra - Will attempt the table extraction from wiki using Beautiful Soup 4

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
page.content

soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

I will use beautiful soup to scan through the table and find the headings I am interested in. This is to verify that BeautifulSoup has found what I am looking for

In [ ]:
# Can also use soup.find(class_='wikitable sortable')
table = soup.find_all(class_='sortable')

for tab in table:
    ths = tab.find_all('th')
    headings = [th.get_text() for th in ths]
    if headings[0:3] == ['Postal Code','Borough','Neighborhood']:
        break
headings

Now I will pass this to a dataframe. Note that p.read_html creates a list of daaframes, meaning that for this table, we only select index 0

In [ ]:
df_wiki_table = pd.read_html(str(table))
df_wiki_table[0]

In [ ]:
# Write out the table as a csv as a checkpoint file
base_filename = 'Canada_Postal_Codes'
csvs_sht = os.path.join(dir_name, base_filename + "." + filename_suffix)
df_wiki_table[0].to_csv(csvs_sht, index = False, header=True)
print ("Final File Extract Produced")

In [ ]:
count = 'Completed Process'
elapsed = (time.time() - start)
print ("%s in %s seconds" % (count,elapsed))